In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
#train_data_dir = '/content/drive/MyDrive/gender_dataset_full/gender_full_body'
#val_data_dir = '/content/drive/MyDrive/gender_dataset_full/gender_full_body'
train_data_dir = '/content/drive/MyDrive/Dataset'
val_data_dir = '/content/drive/MyDrive/Dataset'

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(train_data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 6),  # Two output nodes for male and female 2->6
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 50  #7->50
save_model_path = '/content/drive/MyDrive/components_new.pth'
onnx_model_path = '/content/drive/MyDrive/components_new.onnx'
#save_model_path = '/content/drive/MyDrive/second_trial_new.pth'
#onnx_model_path = '/content/drive/MyDrive/second_trial_new.onnx'  # Specify the path for saving the ONNX model
best_val_loss = float('inf')
early_stopping_counter = 0
patience = 3  # You can adjust this value based on your preference for early stopping

for epoch in range(1, num_epochs + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            exp_lr_scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Epoch {epoch}/{num_epochs} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val':
            if epoch % 5 == 0:  # Save the model every 5th epoch
                torch.save(model.state_dict(), f'/content/drive/MyDrive/Dataset/checkpoints/checkpoint2_{epoch}.pth')

            # Early stopping check
            if epoch > 1 and epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                early_stopping_counter = 0
            else:
                early_stopping_counter += 1

            if early_stopping_counter >= patience:
                print("Early stopping triggered")
                break

# Save the final model
torch.save(model.state_dict(), save_model_path)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Epoch 1/50 Loss: 1.2104 Acc: 0.5920
val Epoch 1/50 Loss: 2.0064 Acc: 0.4256


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Epoch 2/50 Loss: 1.1016 Acc: 0.6316
val Epoch 2/50 Loss: 1.7535 Acc: 0.4414


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

train Epoch 48/50 Loss: 0.6948 Acc: 0.7640
val Epoch 48/50 Loss: 1.0634 Acc: 0.5941
Early stopping triggered


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Epoch 49/50 Loss: 0.7060 Acc: 0.7542
val Epoch 49/50 Loss: 1.0694 Acc: 0.5921
Early stopping triggered


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Epoch 50/50 Loss: 0.7124 Acc: 0.7439
val Epoch 50/50 Loss: 1.0708 Acc: 0.5872
Early stopping triggered


In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 14.8 MB/s eta 0:00:00


In [ ]:
# Export the model to ONNX format
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Input size should match your network's input size
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True)


In [ ]:

checkpoint_path = '/content/drive/MyDrive/gender_dataset_full/checkpoints/checkpoint2_5.pth'
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(torch.load(f'/content/drive/MyDrive/gender_dataset_full/checkpoints/checkpoint2_5.pth'))
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #exp_lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
    #best_val_loss = checkpoint['best_val_loss']
    start_epoch = 5
else:
    start_epoch = 0
    best_val_loss = float('inf')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 15  # Increased the number of epochs for demonstration
save_model_path = '/content/drive/MyDrive/second_trial_new.pth'
onnx_model_path = '/content/drive/MyDrive/second_trial_new.onnx'  # Specify the path for saving the ONNX model
best_val_loss = float('inf')
early_stopping_counter = 0
patience = 3  # You can adjust this value based on your preference for early stopping

for epoch in range(start_epoch, start_epoch + num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            exp_lr_scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Epoch {epoch}/{start_epoch + num_epochs - 1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val':
            # Save the model every 5th epoch
            #if epoch % 5 == 0:
            #    torch.save({
            #        'epoch': epoch,
            #        'model_state_dict': model.state_dict(),
            #        'optimizer_state_dict': optimizer.state_dict(),
            #        'scheduler_state_dict': exp_lr_scheduler.state_dict(),
            #        'best_val_loss': best_val_loss
            #    }, f'gender_classification_model_epoch{epoch}.pth')

            # Early stopping check
            if epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
            else:
                patience -= 1

            if patience == 0:
                print("Early stopping triggered")
                break
            if (epoch + 1) % 1 == 0:
                torch.save(model.state_dict(), f'/content/drive/MyDrive/gender_dataset_full/checkpoints/checkpoint2_{epoch+1}.pth')


# Save the final model
torch.save(model.state_dict(), save_model_path)

train Epoch 5/19 Loss: 0.3352 Acc: 0.8471
val Epoch 5/19 Loss: 0.2157 Acc: 0.9082
train Epoch 6/19 Loss: 0.3085 Acc: 0.8554
val Epoch 6/19 Loss: 0.2467 Acc: 0.8869
train Epoch 7/19 Loss: 0.3075 Acc: 0.8601
val Epoch 7/19 Loss: 0.1996 Acc: 0.9104
train Epoch 8/19 Loss: 0.2829 Acc: 0.8717
val Epoch 8/19 Loss: 0.1899 Acc: 0.9180
train Epoch 9/19 Loss: 0.2784 Acc: 0.8722
val Epoch 9/19 Loss: 0.2045 Acc: 0.9130
train Epoch 10/19 Loss: 0.2748 Acc: 0.8802
val Epoch 10/19 Loss: 0.1779 Acc: 0.9203
train Epoch 11/19 Loss: 0.2605 Acc: 0.8811
val Epoch 11/19 Loss: 0.1739 Acc: 0.9281
train Epoch 12/19 Loss: 0.2204 Acc: 0.9050
val Epoch 12/19 Loss: 0.1442 Acc: 0.9377
train Epoch 13/19 Loss: 0.1973 Acc: 0.9165
val Epoch 13/19 Loss: 0.1385 Acc: 0.9402
train Epoch 14/19 Loss: 0.1887 Acc: 0.9172
val Epoch 14/19 Loss: 0.1348 Acc: 0.9467
train Epoch 15/19 Loss: 0.1908 Acc: 0.9197
val Epoch 15/19 Loss: 0.1367 Acc: 0.9427
Early stopping triggered
train Epoch 16/19 Loss: 0.1853 Acc: 0.9181
val Epoch 16/19 Lo

OnnxExporterError: ignored

In [ ]:
!pip install onnx
import onnx

# Export the model to ONNX format
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Input size should match your network's input size
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True)